In [110]:
import os


In [111]:
# counter = {}
replacement = {}
vocab_size = 1024
merge_num = 1024-256

In [112]:
with open('./small.txt', 'r') as f:
    lines = f.readlines()
    raw_text = '\n'.join(lines)

print(len(raw_text))

32545


In [113]:
text_bytes = raw_text.encode("utf-8") # raw bytes
ids = list(text_bytes) # list of integers in range 0..255

original_ids = ids.copy()

print(text_bytes[:10])

b'\n\nIn Augus'


In [114]:
def get_stats(ids, counter={}):

    min_cnt = float("inf")
    ret = None
    if len(ids) < 2:
        return None
    
    for pair in zip(ids,ids[1:]):
        counter[pair] = counter.get(pair, 0) + 1
        
    # can be more efficient and elegant
    # for pair,val in counter.items():
    #     if(type(val) != type(1) ):
    #         print(val)
    #     if min_cnt > val:
    #         min_cnt = val
    #         ret = pair
    
    ret = max(counter, key=counter.get)
    
    return ret,counter

In [115]:
get_stats(ids)
# print(counter)

((101, 32),
 {(10, 10): 77,
  (10, 73): 8,
  (73, 110): 23,
  (110, 32): 325,
  (32, 65): 129,
  (65, 117): 9,
  (117, 103): 10,
  (103, 117): 12,
  (117, 115): 105,
  (115, 116): 242,
  (116, 32): 372,
  (32, 50): 105,
  (50, 48): 146,
  (48, 49): 61,
  (49, 50): 18,
  (50, 44): 5,
  (44, 32): 294,
  (32, 83): 195,
  (83, 119): 134,
  (119, 105): 206,
  (105, 102): 160,
  (102, 116): 158,
  (32, 114): 138,
  (114, 101): 324,
  (101, 108): 99,
  (108, 101): 164,
  (101, 97): 126,
  (97, 115): 163,
  (115, 101): 114,
  (101, 100): 321,
  (100, 32): 584,
  (32, 34): 122,
  (34, 87): 4,
  (87, 101): 11,
  (101, 32): 701,
  (65, 114): 9,
  (32, 78): 34,
  (78, 101): 14,
  (101, 118): 31,
  (118, 101): 153,
  (101, 114): 487,
  (114, 32): 385,
  (32, 69): 31,
  (69, 118): 9,
  (32, 71): 42,
  (71, 101): 4,
  (101, 116): 45,
  (116, 116): 24,
  (116, 105): 263,
  (105, 110): 454,
  (110, 103): 272,
  (103, 32): 180,
  (32, 66): 58,
  (66, 97): 10,
  (97, 99): 78,
  (99, 107): 32,
  (107, 32)

In [116]:
def merge(ids, pair, new_id):
    new_ids = []
    i = 0

    while i < len(ids) :
        
        if i< len(ids) -1 and pair[0] == ids[i] and pair[1] == ids[i+1]:
            new_ids.append(new_id)
            i += 2
        else: 
            new_ids.append(ids[i])
            i += 1
    
    global replacement
    replacement[new_id] = pair
    
    return new_ids
    

In [117]:
# get_stats(merge(ids,(188, 191), 256))

In [118]:
print(len(ids))
print(len(replacement))

32565
0


In [119]:
while merge_num > 0:
    pair,_ = get_stats(ids)
    
    ids = merge(ids,pair,vocab_size-merge_num)
    merge_num -= 1
    # print("1",end='\t')

In [120]:
print(len(ids))
print(len(replacement))

15479
768


In [121]:
def decode(ids, replacement):

    for i in range(vocab_size,255,-1):
        new_ids = []
        for id in ids:
            if id == i:
                new_ids.append(replacement[i][0])
                new_ids.append(replacement[i][1])
            else:
                new_ids.append(id)
        ids = new_ids
    return ids

In [122]:
final_ids = decode(ids, replacement)

print(len(original_ids))
print(len(final_ids))

for(i,j) in zip(original_ids,final_ids):
    if i != j:
        print(i,j)
        break


import editdistance
# print(editdistance.eval(original_ids, final_ids))

32565
32565
